In [1]:
import argparse
import os
import random
import shutil
import time
import warnings
import sys
sys.path.append('../')

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
# import torchvision.models as models
from resnet import *

from main import *

model_names = sorted(name for name in models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(models.__dict__[name]))

In [2]:
# args = parser.parse_args(args=[])
# args = parser.parse_args()
import easydict 
args = easydict.EasyDict({ "batch-size": 256, 
                          "epochs": 50, 
                          "data": 0, 
                          'arch':'resnet18',
                          'lr':0.1,
                         'momentum':0.9,
                         'weight_decay':1e-4,
                         'start_epoch':0,
                         'gpu':0})


In [3]:
ngpus_per_node = torch.cuda.device_count()
print(ngpus_per_node)
# device = 'cpu'
device = 'cuda'

1


In [4]:
# import numpy as np
# imagenet_embeding = np.load('../data/imagenet_embeding.npy')
# imagenet_embeding.shape

In [5]:
# imagenet_embeding = torch.tensor(imagenet_embeding)
# imagenet_embeding.shape

In [6]:
print("=> using pre-trained model '{}'".format('resnet18'))
# model = models.__dict__['resnet18'](pretrained=True)
# model = models.resnet18(pretrained=False)
model = resnet18(pretrained=False)
model.to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), args.lr,
                            momentum=args.momentum,
                            weight_decay=args.weight_decay)
# optimizer = torch.optim.Adam(
#     model.parameters(), lr=0.1)


=> using pre-trained model 'resnet18'


In [7]:
# model_dict = model.state_dict() 
# for k in model_dict :
#     if 'fc.weight' in k :
#         print(model_dict[k].shape)
#         print(model_dict[k])
#         model_dict[k] = imagenet_embeding
# #     if 'fc.bias' in k :
# #         print(model_dict[k])        
# #     print(model_dict[k].shape)
# model.load_state_dict(model_dict)
# model_dict = model.state_dict() 
# for k in model_dict :
#     if 'fc.weight' in k :
# #         print(model_dict[k].shape)
#         print(model_dict[k])

In [8]:
# model.fc.weight.requires_grad = False

In [9]:
# for p in model.parameters() :
#     print(p)

In [10]:
# model.eval()

In [11]:
# Data loading code
data_dir = '../ILSVRC/Data/CLS-LOC/'
traindir = os.path.join(data_dir, 'train')
valdir = os.path.join(data_dir, 'val')

In [12]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
val_dataset = datasets.ImageFolder(valdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]))

In [13]:
# next(iter(train_dataset))

In [14]:
# train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
train_sampler = None

In [15]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=256, shuffle=(train_sampler is None),
    num_workers=8, pin_memory=True, sampler=train_sampler)

In [16]:
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=32, shuffle=False,
    num_workers=4, pin_memory=True)

In [17]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [18]:
best_acc1 = 0
acc1 = 0
train_loss = []
val_acc = []
for epoch in range(args.start_epoch, args.epochs):
    adjust_learning_rate(optimizer, epoch, args)

    # train for one epoch
    epoch_loss = train(train_loader, model, criterion, optimizer, epoch, args)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion, args)  
    
    train_loss.append(epoch_loss)
    val_acc.append(acc1)
    print('************train_loss {} val_acc {}*************'.format(epoch_loss, acc1))
    
    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)

#     if not args.multiprocessing_distributed or (args.multiprocessing_distributed
#             and args.rank % ngpus_per_node == 0):
    save_checkpoint({
        'epoch': epoch + 1,
        'arch': args.arch,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)    

Epoch: [0][   0/5005]	Time  2.572 ( 2.572)	Data  2.105 ( 2.105)	Loss 7.0025e+00 (7.0025e+00)	Acc@1   0.39 (  0.39)	Acc@5   1.17 (  1.17)
Epoch: [0][1001/5005]	Time  0.394 ( 0.389)	Data  0.000 ( 0.003)	Loss 5.5999e+00 (6.1512e+00)	Acc@1   6.25 (  2.20)	Acc@5  17.97 (  7.43)
Epoch: [0][2002/5005]	Time  0.397 ( 0.391)	Data  0.000 ( 0.002)	Loss 4.8257e+00 (5.6935e+00)	Acc@1  12.50 (  4.67)	Acc@5  29.30 ( 13.55)
Epoch: [0][3003/5005]	Time  0.393 ( 0.392)	Data  0.000 ( 0.001)	Loss 4.4676e+00 (5.3463e+00)	Acc@1  14.06 (  7.35)	Acc@5  33.59 ( 19.15)
Epoch: [0][4004/5005]	Time  0.387 ( 0.392)	Data  0.000 ( 0.001)	Loss 4.1200e+00 (5.0676e+00)	Acc@1  20.31 (  9.94)	Acc@5  43.75 ( 23.95)
 * Acc@1 22.219 Acc@5 45.560
************train_loss 4.839285239496908 val_acc 22.21889305114746*************
Epoch: [1][   0/5005]	Time  2.472 ( 2.472)	Data  2.325 ( 2.325)	Loss 3.8029e+00 (3.8029e+00)	Acc@1  21.09 ( 21.09)	Acc@5  43.36 ( 43.36)
Epoch: [1][1001/5005]	Time  0.389 ( 0.396)	Data  0.000 ( 0.003)	Loss 